In [30]:
import S2VT
import readin_raw as read
import torch
import numpy as np

In [31]:
input_size = 4096
batch_size = 64
encoder_hidden = 256
decoder_hidden = 256
encoder_layer = 1
decoder_layer = 1

In [32]:
training_data, one_hot_size, answer_len = read.generate_dataloader(batch_size = batch_size)

Loading Data
1450
Finish count words
Process data  0  to list
Process data  1  to list
Process data  2  to list
Process data  3  to list
Process data  4  to list
Process data  5  to list
Process data  6  to list
Process data  7  to list
Process data  8  to list
Process data  9  to list
Process data  10  to list
Process data  11  to list
Process data  12  to list
Process data  13  to list
Process data  14  to list
Process data  15  to list
Process data  16  to list
Process data  17  to list
Process data  18  to list
Process data  19  to list
Process data  20  to list
Process data  21  to list
Process data  22  to list
Process data  23  to list
Process data  24  to list
Process data  25  to list
Process data  26  to list
Process data  27  to list
Process data  28  to list
Process data  29  to list
Process data  30  to list
Process data  31  to list
Process data  32  to list
Process data  33  to list
Process data  34  to list
Process data  35  to list
Process data  36  to list
Process dat

In [33]:


output_dim = one_hot_size

learning_rate = 0.001
training_epoch = 200
cuda_avilabel = True
# we can change the schedule sampling rate by parameter lamda; the first 100 epochs set the rate to 1
lamda = 0.998

In [37]:
def training(input_size,batch_size,encoder_hidden,decoder_hidden,encoder_layer,decoder_layer,output_dim,cuda_avilabel,lamda,training_data,load_model = "", optim_state = ""):
    model = None
    if load_model == "":
        model = S2VT.S2VT(input_size,batch_size,encoder_hidden,decoder_hidden,encoder_layer,decoder_layer,output_dim, False)
    else:
        model = torch.load(load_model)
    if(cuda_avilabel):
        model = model.cuda()
    print(model)
    optimizer = None
    if optim_state == "":
        optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)
    else:
        state = torch.load(optim_state)
        optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)
        optimizer.load_state_dict(state)
    criterion = torch.nn.CrossEntropyLoss()
    loss_array = []
    optimizer.zero_grad()
    
    for epoch in range(training_epoch):
        print("epoch: ",epoch)
        epoch_loss = 0
        for x,y,z in training_data:
            encoder_input = x  # the input to encoder during encoding stage
            correct_answer = y   # correct answer 
	        #encoder_pad = torch.zeros((correct_answer[0],correct_answer[1],input_size),dtype = torch.float)      # the input to encoder during decoding stage
            _, target = correct_answer.max(dim=2)  # not one-hot, but a number imply the right class
            
            if(cuda_avilabel):
                encoder_input = encoder_input.transpose(1, 0).cuda()
                correct_answer = correct_answer.transpose(1, 0).cuda()
	        	#encoder_pad = encoder_pad.cuda()
                target = target.cuda()
            if correct_answer.shape[1] != batch_size:
                model.encoder_h = torch.zeros((encoder_layer, correct_answer.shape[1], encoder_hidden),
                                      dtype = torch.float32).cuda()
                model.encoder_c = torch.zeros((encoder_layer, correct_answer.shape[1], encoder_hidden),
                                              dtype = torch.float32).cuda()
                model.decoder_h = torch.zeros((decoder_layer, correct_answer.shape[1], decoder_hidden),
                                              dtype = torch.float32).cuda()
                model.decoder_c = torch.zeros((decoder_layer, correct_answer.shape[1], decoder_hidden),
                                              dtype = torch.float32).cuda()
                model.batch_size = correct_answer.shape[1]
            else:
                model.encoder_h = torch.zeros((encoder_layer, batch_size, encoder_hidden),
                                      dtype = torch.float32).cuda()
                model.encoder_c = torch.zeros((encoder_layer, batch_size, encoder_hidden),
                                              dtype = torch.float32).cuda()
                model.decoder_h = torch.zeros((decoder_layer, batch_size, decoder_hidden),
                                              dtype = torch.float32).cuda()
                model.decoder_c = torch.zeros((decoder_layer, batch_size, decoder_hidden),
                                              dtype = torch.float32).cuda()
                model.batch_size = batch_size
	        # target = test_correct_answer[iteration] # not one-hot, but a number imply the right class
            output = model(encoder_input,correct_answer, answer_len, z) # correct_answer is just a parameter but not used during encoding stage
	        # eocoded can be used for attention
	        #model.encoder_stage = False
	        #model.decoder_stage = True
	        #output = model(encoder_pad,correct_answer)
            #print(target[0,:z[0].item()])
            
            
            loss = None
            output_tmp = torch.stack(output) #(seq_len,batch,one_hot)
            #print(len(output))
            #print(output_tmp.shape)
            #print(output_tmp[:z[0].item(),0].shape)
            #break
            
            for data_num in range(z.shape[0]):
                length = z[data_num].item()
                if type(loss) == type(None):
                    loss = criterion(output_tmp[:length,data_num], target[data_num,:length])
                else :
                    loss += criterion(output_tmp[:length,data_num], target[data_num,:length])
            '''
            output_tmp = torch.cat((output),0)
            output = output_tmp.view(-1,output_dim)
            target = target.view(-1)
            loss = None
            position = 0
            for data_num in range(z.shape[0]):
                length = z[data_num].item()
                if type(loss) == type(None):
                    loss = criterion(output_tmp[position:position+length], target[position:position+length])
                else :
                    loss += criterion(output_tmp[position:position+length], target[position:position+length])
                position += answer_len # answer_len is the max length for a sentence
            '''
            
            '''
            print(output_tmp[0][:9].shape)
            output = output_tmp.view(-1,output_dim)
            print(output.shape)
            target = target.view(-1)
            loss = criterion(output,target)
            '''
            #print(loss.item())
            epoch_loss += loss.item()
            loss_array.append(loss.item())
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(epoch_loss / (1450//64 + 1))
        # we can change the schedule sampling rate by parameter lamda; the first 100 epochs set the rate to 1
        if epoch >= 50:
            tmp_rate = model.schedule_sample_rate
            model.schedule_sample_rate = lamda * tmp_rate
    ### save model ###
    torch.save(model,'s2vt_1025_final.pkl')
    torch.save(optimizer.state_dict(), "s2vt_1025_optim.pkl")
    return loss_array

def restore_model(model_name):
	### restore model ###
	net = torch.load(model_name)
	print(net)
	return net

In [ ]:

history = training(input_size=input_size, batch_size=batch_size, encoder_hidden=encoder_hidden,
                   decoder_hidden=decoder_hidden, encoder_layer=encoder_layer, decoder_layer=decoder_layer,
                   output_dim=one_hot_size, cuda_avilabel=True, lamda=lamda, training_data = training_data)

for i in range(5):
    training_data, one_hot_size, answer_len = read.generate_dataloader(batch_size = batch_size)
    history = training(input_size=input_size, batch_size=batch_size, encoder_hidden=encoder_hidden,
                       decoder_hidden=decoder_hidden, encoder_layer=encoder_layer, decoder_layer=decoder_layer,
                       output_dim=one_hot_size, cuda_avilabel=True, lamda=lamda, training_data = training_data,load_model = "s2vt_1025_final.pkl",optim_state = "s2vt_1025_optim.pkl")



S2VT(
  (output_softmax): Softmax()
  (encoder): LSTM(4096, 256)
  (decoder): LSTM(512, 256)
  (input_embedding): Linear(in_features=2890, out_features=256, bias=True)
  (output_embedding): Linear(in_features=256, out_features=2890, bias=True)
  (attn): Attention(
    (dropout): Dropout(p=0.2)
    (linear_l1du_max): Linear(in_features=512, out_features=80, bias=True)
    (linear_2du_du): Linear(in_features=512, out_features=256, bias=True)
    (softmax): Softmax()
    (relu): PReLU(num_parameters=1)
  )
)
epoch:  0
367.3603701384171
epoch:  1
294.673083761464
epoch:  2
283.0578215225883
epoch:  3
272.1202339504076
epoch:  4
258.49542833411175
epoch:  5
240.87453891920006
epoch:  6
223.479608286982
epoch:  7
207.19445071013078
epoch:  8
191.3580972422724
epoch:  9
176.66564576522163
epoch:  10
163.36411219057828
epoch:  11
151.4398020868716
epoch:  12
140.23004249904466
epoch:  13
130.2040856402853
epoch:  14
120.64184736168903
epoch:  15
111.88522571066152
epoch:  16
103.80846968941067

In [ ]:
a=0

In [48]:
for x,y,z in training_data:
    print(z[0])
    break
    
    

tensor([8])


In [9]:
a

1450

In [11]:
answer_len

44